In [1]:
import numpy as np

# SBEND Benchmarking

Default uses arc length of 0.5 m and bend angle of $\pi / 2$. The arc length is entered directly into elegant for SBEND.L while OPAL the chord length is used to set SBEND.L (calculated from the arc length and bend angle).

From SLAC-75 for a dipole:

$R_{11} = cos(k_x L)$

$R_{12} = -k_x sin(k_x L)$

Where $k_x = \sqrt{1-n} / R$, $R$ is the bending radius, $L$ is the arc length, and $n$ is the magnet index which is normally $0$ for pure dipole. 

For our settings this should give easy to work with values of

$R_{11} = 0$

$R_{12} = -\pi$

### Element Settings

In [2]:
arc_length = 0.5
bend_angle = np.pi / 2.
bend_radius = arc_length / bend_angle

chord_length = 2 * bend_radius * np.sin(bend_angle / 2.)

In [3]:
# Arc length if use a chord length of 0.5
# 0.5 / (2  * np.sin(bend_angle / 2.)) * bend_angle

In [4]:
def r11_dipole(radius, length):
    n = 0
    kx = np.sqrt(1 - n) / radius
    R11 = np.cos(kx * length)
    
    return R11

def r12_dipole(radius, length):
    n = 0
    kx = np.sqrt(1 - n) / radius
    R12 = -kx * np.sin(kx * length)
    
    return R12

### Verify calculation

In [5]:
r11_dipole(bend_radius, arc_length)

6.123233995736766e-17

In [6]:
r12_dipole(bend_radius, arc_length)

-3.141592653589793

# Distribution

Both simulations use a single particle with coordinates:

x, xp, y, yp, t, p = 1e-3, 0, 0, 0, 0, 273.97134

The transverse momenta units differ between elegant and OPAL but are initially zero so it should not matter. Other units are the same bewteen the two codes.

The final result should be immediately after the dipole edge face:

$x_f = R_{11} * x_0 + R_{12} x'_0 = 0.$

$x'_f = R_{11} * x_0 + R_{12} x'_0 = \pi \, \times 10^{-3}$

# OPAL

OPAL simulation uses very small time step (1 fs) to get good accuracy. Some of this may be down to the 'almost' hard edge field on the dipole that needs to be integrated through as accurately as possible.

The monitor is also not positioned exactly at the dipole exit right now so the $x_f$ value may not be close to 0. Positioning the monitor downstream of the dipole still allows for verifying $x'_f$ though.

#### Run

In [8]:
! opal opal.in

Ippl> CommMPI: Parent process waiting for children ...
Ippl> CommMPI: Initialization complete.
>                ____  _____       ___ 
>               / __ \|  __ \ /\   | | 
>              | |  | | |__) /  \  | |
>              | |  | |  ___/ /\ \ | |
>              | |__| | |  / ____ \| |____
>               \____/|_| /_/    \_\______|
OPAL> 
OPAL> This is OPAL (Object Oriented Parallel Accelerator Library) Version 2.2.0
OPAL>                                 git rev. 
OPAL> 
OPAL> 
OPAL>                      (c) PSI, http://amas.web.psi.ch
OPAL> 
OPAL> 
OPAL> The optimiser (former opt-Pilot) is integrated 
OPAL> 
OPAL> Please send cookies, goodies or other motivations (wine and beer ... ) 
OPAL> to the OPAL developers opal@lists.psi.ch
OPAL> 
OPAL> Time: 18:30:16 date: 02/09/2020
OPAL> 
OPAL> Couldn't find startup file "/home/vagrant/init.opal".
OPAL> Note: this is not mandatory for an OPAL simulation!
OPAL> 
OPAL> * Reading input stream "opal.in".
OPAL> 
OPAL> value: {GAMMA*BETA,EMA

In [9]:
import h5py as h5

#### Result

In [10]:
test = h5.File('opal_monitor_dipole_end.h5', 'r')

In [11]:
for key in test:
    print(key)

Attachment
Step#0


In [12]:
# Final x momentum in beta*gamma
test['Step#0/px'][()] 

array([-0.86053185])

In [13]:
# Final x momentum angle
test['Step#0/px'][()]  / test['Step#0/pz'][()]

array([-0.00314097])

In [14]:
# Deviation from expected
test['Step#0/px'][()]  / test['Step#0/pz'][()] / 1e-3 - r12_dipole(bend_radius, arc_length)

array([0.00062142])

# ELEGANT

In [15]:
from rsbeams.rsdata.SDDS import writeSDDS, readSDDS

#### Create distribution if necessary

In [22]:
new_distr = writeSDDS()

new_distr.create_column('x', [1e-3,], 'double', colUnits='m')
new_distr.create_column('xp', [0.,], 'double', colUnits='')
new_distr.create_column('y',  [0.,], 'double', colUnits='m')
new_distr.create_column('yp',  [0.,], 'double', colUnits='')
new_distr.create_column('t',  [0.,], 'double', colUnits='s')
new_distr.create_column('p',  [272.973,], 'double', colUnits='')

new_distr.create_parameter('particles', 1, 'long')
new_distr.create_parameter('pCentral', 272.973, 'double', parUnits='MeV/c')

new_distr.save_sdds('bunch_elegant.sdds', dataMode='binary')

#### Run

In [23]:
! python sbend.py

Running elegant at Wed Sep  2 19:56:51 2020

This is elegant 2019.3.0, May 30 2020, by M. Borland, J. Calvey, M. Carla', N. Carmignani, M. Ehrlichman, L. Emery, W. Guo, R. Lindberg, V. Sajaev, R. Soliday, Y.-P. Sun, C.-X. Wang, Y. Wang, Y. Wu, and A. Xiao.
Thanks for using elegant.  Please cite the following reference in your publications:
  M. Borland, "elegant: A Flexible SDDS-Compliant Code for Accelerator Simulation,"
  Advanced Photon Source LS-287, September 2000.
If you use a modified version, please indicate this in all publications.
Link date: May 30 2020 20:36:55, SVN revision: unknown
statistics:    ET:     00:00:00 CP:    0.01 BIO:0 DIO:0 PF:0 MEM:5662
&global_settings
    inhibit_fsync = 0,
    echo_namelists = 1,
    mpi_randomization_mode = 3,
    exact_normalized_emittance = 0,
    SR_gaussian_limit = 3.000000000000000e+00,
    inhibit_seed_permutation = 0,
    log_file = {NULL},
    error_log_file = {NULL},
    share_tracking_based_matrices = 1,
    parallel_tracking_b

#### Result

In [24]:
elegant_out = readSDDS('W1.filename.sdds')
elegant_out.read()

&description &description text="watch-point phase space--input: elegant.ele  lattice: elegant.lte", contents="watch-point phase space", &end
 not parsed
Could not read page 1


In [25]:
# Final x momentum as angle
elegant_out.columns['xp']

array([[-0.00314159]])

In [26]:
# Deviation from expected result
elegant_out.columns['xp'] / 1e-3 - r12_dipole(bend_radius, arc_length)

array([[0.]])